In [13]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import csv 
import time

def get_job_id(job_name):
    t1=time.time()
    baseurl='https://hk.jobsdb.com/hk/search-jobs/'+re.sub(' ','-',job_name)+'/1'
    
    r=requests.get(baseurl,headers={"User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1"}).text
    soup=BeautifulSoup(r,'lxml')
    
    pattern=re.compile(r'(\d+)')
    total_num_jobs=re.findall('\d+', soup.find('meta', property="og:description")['content'])
    
    #number of page you need to access all jobs
    float(total_num_jobs[0]) 
    num_page=int(total_num_jobs[0]) //30 +1   
    
    global job_id 
    job_id=[]
    
    skills = ['R', 'RStudio', 'Markdown', 'Latex', 'SparkR', 'D3', 'D3.js','Microsoft Office','Excel',
    'Unix', 'Linux', 'MySQL', 'Microsoft SQL server', 'SQL','VBA','Qlik'
    'Python', 'SPSS', 'SAS', 'C#','Matlab','Java', 'keras',
    'JavaScript', 'HTML', 'HTML5', 'CSS', 'CSS3','PHP', 'Excel', 'Tableau',
    'AWS', 'Amazon Web Services ','Google Cloud Platform', 'GCP','theano'
    'Microsoft Azure', 'Azure', 'Hadoop', 'Spark','python'
    'MapReduce', 'Map Reduce','Shark', 'Cassandra',
    'NoSQL', 'MongoDB', 'GIS', 'Haskell', 'Scala', 'Ruby','Perl',
    'Mahout', 'Stata','Deep Learning','Machine Learning',  'Pytorch', "Tensorflow",'Caffe','API','seo','Business Intelligence'
    , 'BI' ]

    skill_lower=[i.lower() for i in skills]
    skill_set=list(set(skill_lower))
    skill_set
    
    
    for i in range(1,num_page+1):
        time.sleep(1)
        print(f'Now start to craw job search result P.{i}')
        newurl=baseurl[:-1]+str(i)
        r=requests.get(newurl,headers={"User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1"}).text
        soup=BeautifulSoup(r,'lxml')
        
        script_str=soup.find_all("script")[1].get_text()[28:]

        pattern=re.compile(r'(https\W\Wu002F\Wu002Fhk\Wjobsdb\Wcom\Wu002Fhk\Wu002Fen\Wu002Fjob\Wu002F)([a-zA-Z0-9]+\W)*') 
        job_url_raw=[]
        job_url_raw2=[]
        for i in pattern.finditer(script_str):
            job_url_raw.append(i[0])

        for i in job_url_raw:
            a=re.sub("u002F",'',i)
            b=re.sub('"','',a)
            c=b.split('\\')[-1]
            print(c)
            job_id.append(c)
        
    for i in job_id:
        try: 
            print(f'Now start to craw individual job id {i}')
 
            joburl='https://hk.jobsdb.com/hk/en/job/'+str(i)
            r=requests.get(joburl,headers={"User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1"}).text
            soup=BeautifulSoup(r,'lxml')

            try:
                job_title= soup.find(class_='general-pos ad-y-auto-txt2').text.strip()
            except:
                job_title= soup.find(class_='general-pos ad-y-auto-txt').text.strip()
            print(f'Job title is {job_title}')

            try:
                company= soup.find(class_='jobad-header-company').text.lstrip()
            except:
                company= soup.find(class_='jobad-header-company ad-y-auto-txt1').text.lstrip()
            print(f'company {job_title}')

            role=[i.text.lower() for i in soup.find_all('span')]

            role2=str(role)
            
            print(f' Got job role for this post {job_title}')
            try:
                yr_exp=soup.find('b', class_='primary-meta-exp').text.strip()
                yr_exp=yr_exp[0]
            except:
                yr_exp='Didnt specify'
            print(f' Exp is {yr_exp} ')

            try:
                level=soup.find('b', class_='primary-meta-lv').text.strip()
            except:
                level='Didnt specify'
            print(f' level is {yr_exp} ')

            job_title_list.append(job_title)        
            company_list.append(company) 
            yr_exp_list.append(yr_exp)
            role_list.append(role)
            
            print('now to matching')
            required_skill=[]
            for i in skill_set:
                result=re.search(i,role2)
                if result:
                    required_skill.append(i)
                    skill_list.append(i)
                else:
                    pass
                
            with open ('jobsDBtest2_csv_result.csv','a') as f:
                csv_writer= csv.writer(f)
                csv_writer.writerow([i,job_title,company,yr_exp,level,required_skill])    
)
        except:
            print('Cant crawd some id')
            pass
    t2= time.time()
    print(f'It takes {t2-t1} to crawd all the jobs info')
    print(f'There are total {len(job_id)} jobs in this category')
    
    frequency={}
    for item in skill_list:
        frequency[item]=frequency.get(item,0)+1
    skill_count=pd.Series(frequency)
    skill_count.sort_values(ascending=False,inplace=True)
    
    return skill_count
    

In [20]:
get_job_id('data')

Now start to craw job search result P.1
data-governance-manager-100003006604752
clinical-data-research-associate-100003006607289
technical-support-data-center-100003006607129
data-analyst-100003006607026
associate-data-analytics-100003006607009
data-analytic-manager-enterprise-data-analytics-and-management-100003006607004
human-resources-assistant-100003006606000
data-analyst-100003006606872
data-entry-clerk-100003006606871
senior-database-administrator-100003006606694
business-intelligence-data-warehouse-100003006606689
database-administrator-100003006606666
master-data-management-assistant-manager-100003006606655
data-systems-manager-100003006606585
data-centre-operator-100003006606308
senior-engineer-engineer-100003006606149
data-analyst-asia-pacific-100003006606013
data-entry-clerks-100003006606012
technical-analyst-assistant-technical-analyst-data-centre-100003006605665
data-scientist-100003006605660
data-center-technical-support-100003006605393
network-technical-officer-technicia

Now start to craw job search result P.7
data-entry-clerk-100003006586823
db2-database-administrator-major-bank-100003006590901
database-administrator-100003006591583
data-analytics-consultant-associate-data-analytics-consultant-100003006591249
data-centre-operator-
specialist-100003006591129
data-analyst-100003006591105
data-analyst-business-planning-manager-100003006591095
data-entry-clerk-100003006590960
facility-engineer-100003006590554
technical-support-officer-100003006590505
system-engineer-100003006590402
solution-architecture-100003006590404
software-engineer-software-engineer-trainee-100003006590403
system-engineer-100003006590405
pre-sales-engineer-100003006590406
account-manager-account-executive-100003006590400
account-manager-account-executive-100003006590401
customer-services-representative-100003006590386
warehouse-supervisor-100003006590387

database-administrator-oracle-100003006587518
senior-analyst-analyst-100003006589983
marketing-manager-customer-data-management-10

Now start to craw job search result P.13

senior-business-strategy-manager-business-strategy-manager-100003006575025
project-manager-100003006563145
data-architect-leading-financial-institution-100003006574865
account-manager-100003006574798
analyst-programmer-system-analyst-100003006570225
database-administrator-100003006570217
system-analyst-analyst-programmer-software-engineer-100003006574670
data-scientists-ai-developer-and-researcher-big-data-engineer-100003006574668
market-data-specialist-financial-services-100003006574659
specialist-engineer-operator-sa-and-ap-100003006574616
part-time-data-entry-clerk-100003006574581
data-speech-analytics-100003006574358
data-engineer-100003006572765
data-centre-officer-manager-bank-100003006571815
it-and-data-center-support-100003006570566
data-entry-clerk-100003006569099
online-marketing-specialist-100003006567262
web-designer-100003006567239
major-bank-data-warehouse-bi-sa-ap-p-100003006573796
oracle-database-administrator-major-bank-1000030

Now start to craw job search result P.19
data-analyst-100003006551082
senior-it-infrastructure-specialist-100003006552578
database-administrator-senior-system-specialist-it-manager-database-100003006552555
data-standards-manager-project-coordinator-100003006552472
data-market-analyst-100003006552430
data-analyst-market-analyst-100003006552243
database-administrator-100003006552210
data-center-operator-100003006552075
data-team-intern-100003006551934
data-specialist-100003006551914
application-developer-data-system-100003006551923
part-time-data-entry-marketing-department-100003006551865
digital-project-coordinator-100003006550645
technical-consultant-database-100003006551711
cloud-data-business-analyst-100003006551605
lead-data-architect-100003006551260
data-remediation-analyst-retail-banking-and-wealth-management-100003006551148
data-engineer-100003006550897
2018-global-data-market-data-analyst-hong-kong-100003006550740
senior-data-scientist-100003006550693
senior-data-engineer-data-a

marketing-manager-finance-data-b2c-base-100003006530908
full-stack-web-developer-100003006530055
junior-multimedia-designer-100003006530080
insights-centre-lead-data-and-analytics-it-advisory-100003006520901
data-entry-clerk-
property-strategy-data-analytics-100003006530542
senior-data-scientist-100003006530512
data-engineer-cyber-security-100003006530510
senior-data-engineer-data-architect-100003006530505
data-entry-clerk-100003006530460
director-data-engineering-100003006530421
senior-data-entry-clerk-100003006530285
data-analyst-customer-data-analyst-marketing-data-analyst-listed-co-100003006530061
market-data-specialist-financial-services-100003006529846
senior-analyst-programmer-100003006529777
marketing-communication-executive-100003006529562
Now start to craw job search result P.25
senior-database-administrator-it-operations-department-100003006529377

system-database-administrator-chong-hing-securities-100003006529289
data-analyst-100003006529120
business-development-manager-10

...

Job title is Project Manager/ Specialist & Engineer (Datacenter/ Middleware/ Mail/ Window/Unix) / DBA / Operator
company Project Manager/ Specialist & Engineer (Datacenter/ Middleware/ Mail/ Window/Unix) / DBA / Operator
 Got job role for this post Project Manager/ Specialist & Engineer (Datacenter/ Middleware/ Mail/ Window/Unix) / DBA / Operator
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id big-data-developer-data-scientist-100003006541127
Job title is Big data developer / Data Scientist - up to 35 - 45k
company Big data developer / Data Scientist - up to 35 - 45k
 Got job role for this post Big data developer / Data Scientist - up to 35 - 45k
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id customer-data-analyst-100003006540998
Job title is Customer Data Analyst
company Customer Data Analyst
 Got job role for this post Customer Data Analyst
 Exp is 2 
 level is 2 
now to matching


Job title is Security Analyst / Security Engineer – Data Center Operation
company Security Analyst / Security Engineer – Data Center Operation
 Got job role for this post Security Analyst / Security Engineer – Data Center Operation
 Exp is 3 
 level is 3 
now to matching
Now start to craw individual job id oracle-database-administrator-major-bank-100003006534401
Job title is Oracle Database Administrator (DBA) – Major Bank
company Oracle Database Administrator (DBA) – Major Bank
 Got job role for this post Oracle Database Administrator (DBA) – Major Bank
 Exp is 2 
 level is 2 
now to matching
Now start to craw individual job id core-business-services-finance-finance-system-and-data-analyst-100003006539569
Job title is Core Business Services – Finance – Finance System and Data Analyst
company Core Business Services – Finance – Finance System and Data Analyst
 Got job role for this post Core Business Services – Finance – Finance System and Data Analyst
 Exp is 4 
 level is 4 
now to mat

Job title is IT Officer(Data, cybersecurity)
company IT Officer(Data, cybersecurity)
 Got job role for this post IT Officer(Data, cybersecurity)
 Exp is 2 
 level is 2 
now to matching
Now start to craw individual job id marketing-executive-marketing-manager-100003006534629
Job title is Marketing Executive/ Marketing Manager
company Marketing Executive/ Marketing Manager
 Got job role for this post Marketing Executive/ Marketing Manager
 Exp is 2 
 level is 2 
now to matching
Now start to craw individual job id data-centre-operator-100003006534555
Job title is Data Centre Operator (Facilities Management)
company Data Centre Operator (Facilities Management)
 Got job role for this post Data Centre Operator (Facilities Management)
 Exp is 2 
 level is 2 
now to matching
Now start to craw individual job id data-entry-customer-service-clerk-trainee-100003006534421
Job title is Data Entry / Customer Service Clerk TRAINEE ( 日資公司 / Shatin  / 福利齊 )
company Data Entry / Customer Service Clerk TR

Job title is Data Entry Clerk (2 months contract) (PM-DEC-JDB)
company Data Entry Clerk (2 months contract) (PM-DEC-JDB)
 Got job role for this post Data Entry Clerk (2 months contract) (PM-DEC-JDB)
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id director-data-engineering-100003006530421
Job title is Director, Data Engineering
company Director, Data Engineering
 Got job role for this post Director, Data Engineering
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id senior-data-entry-clerk-100003006530285
Job title is Senior / Data Entry Clerk (Billing)
company Senior / Data Entry Clerk (Billing)
 Got job role for this post Senior / Data Entry Clerk (Billing)
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id data-analyst-customer-data-analyst-marketing-data-analyst-listed-co-100003006530061
Job title is Data Analyst / Customer Data Analys

r                        2468
bi                       2177
excel                    1215
sql                      1010
sas                       419
machine learning          369
java                      353
hadoop                    322
api                       314
spark                     254
vba                       240
linux                     238
tableau                   230
business intelligence     228
gis                       210
scala                     209
unix                      164
mysql                     139
html                      131
javascript                115
aws                       107
nosql                      94
css                        72
spss                       68
php                        65
perl                       64
microsoft office           61
c#                         56
tensorflow                 55
deep learning              52
matlab                     52
html5                      50
mongodb                    49
azure     

In [15]:
get_job_id('data analyst')

Now start to craw job search result P.1
senior-database-records-management-analyst-100003006604388
senior-data-analyst-100003006603771
data-business-analyst-100003006603709
big-data-analyst-international-company-100003006603634
senior-campaign-management-executive-100003006603409
associate-data-analyst-100003006602456
senior-data-analyst-100003006602301
data-analyst-data-engineer-digital-platform-insurance-industry-100003006601860
system-analyst-big-data-platform-data-engineering-100003006600193
part-time-data-analyst-assistant-investment-100003006600151
analyst-programmer-programmer-data-warehouse-and-bi-100003006599805
associate-analyst-center-of-innovation-100003006599700
data-analyst-100003006599400
senior-data-analyst-100003006598761
data-analyst-senior-data-analyst-100003006597338
network-data-analyst-100003006596992
big-data-system-analyst-analyst-programmer-software-engineer-100003006596822
data-analyst-100003006596604
assistant-data-analyst-100003006596551
system-analyst-big-d

Job title is Data Business Analyst
company Data Business Analyst
 Got job role for this post Data Business Analyst
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id big-data-analyst-international-company-100003006603634
Job title is Big Data Analyst - International Company
company Big Data Analyst - International Company
 Got job role for this post Big Data Analyst - International Company
 Exp is 3 
 level is 3 
now to matching
Now start to craw individual job id senior-campaign-management-executive-100003006603409
Job title is Senior Campaign Management Executive (Senior Data Analyst)
company Senior Campaign Management Executive (Senior Data Analyst)
 Got job role for this post Senior Campaign Management Executive (Senior Data Analyst)
 Exp is 5 
 level is 5 
now to matching
Now start to craw individual job id associate-data-analyst-100003006602456
Job title is ASSOCIATE - DATA ANALYST
company ASSOCIATE - DATA ANALYST
 Got job role for

Job title is Business Analyst (Data Researcher and Automation Developer)
company Business Analyst (Data Researcher and Automation Developer)
 Got job role for this post Business Analyst (Data Researcher and Automation Developer)
 Exp is N 
 level is N 
now to matching
Now start to craw individual job id analyst-programmer-system-analyst-100003006588823
Job title is Analyst Programmer / System Analyst (MIS, Data Warehouse)
company Analyst Programmer / System Analyst (MIS, Data Warehouse)
 Got job role for this post Analyst Programmer / System Analyst (MIS, Data Warehouse)
 Exp is 3 
 level is 3 
now to matching
Now start to craw individual job id database-analyst-programmer-programmer-100003006588284
Job title is Database Analyst Programmer / Programmer (Ref: JDB/MSU/AP)
company Database Analyst Programmer / Programmer (Ref: JDB/MSU/AP)
 Got job role for this post Database Analyst Programmer / Programmer (Ref: JDB/MSU/AP)
 Exp is 2 
 level is 2 
now to matching
Now start to craw individ

Job title is Data Analyst (Insurtech, A.I., Permanent)
company Data Analyst (Insurtech, A.I., Permanent)
 Got job role for this post Data Analyst (Insurtech, A.I., Permanent)
 Exp is 3 
 level is 3 
now to matching
Now start to craw individual job id business-analyst-data-banking-contract-100003006583298
Job title is Business Analyst - Data 12 months renewable banking contract
company Business Analyst - Data 12 months renewable banking contract
 Got job role for this post Business Analyst - Data 12 months renewable banking contract
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id data-analyst-100003006583009
Job title is Data Analyst (Financial Service)
company Data Analyst (Financial Service)
 Got job role for this post Data Analyst (Financial Service)
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id junior-data-analyst-100003006582610
Job title is Junior Data Analyst (3-month Contrac

Job title is Analyst Programmer/Senior Analyst Programmer (Java+SAS/Perm/banking/datawarehouse)
company Analyst Programmer/Senior Analyst Programmer (Java+SAS/Perm/banking/datawarehouse)
 Got job role for this post Analyst Programmer/Senior Analyst Programmer (Java+SAS/Perm/banking/datawarehouse)
 Exp is 4 
 level is 4 
now to matching
Now start to craw individual job id business-analyst-data-banking-100003006568100
Job title is Business Analyst, Data  - 12 months banking renewable contract (up to 38k)
company Business Analyst, Data  - 12 months banking renewable contract (up to 38k)
 Got job role for this post Business Analyst, Data  - 12 months banking renewable contract (up to 38k)
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id data-analyst-fmcg-100003006568096
Job title is Data Analyst- FMCG
company Data Analyst- FMCG
 Got job role for this post Data Analyst- FMCG
 Exp is 2 
 level is 2 
now to matching
Now start to craw individu

Job title is Data/ Market Analyst [Digital platform/ SQL/ up to 25k]
company Data/ Market Analyst [Digital platform/ SQL/ up to 25k]
 Got job role for this post Data/ Market Analyst [Digital platform/ SQL/ up to 25k]
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id data-analyst-market-analyst-100003006552243
Job title is Data Analyst/ Market Analyst ($18K-$35K)
company Data Analyst/ Market Analyst ($18K-$35K)
 Got job role for this post Data Analyst/ Market Analyst ($18K-$35K)
 Exp is 2 
 level is 2 
now to matching
Now start to craw individual job id cloud-data-business-analyst-100003006551605
Job title is Cloud Data Business Analyst
company Cloud Data Business Analyst
 Got job role for this post Cloud Data Business Analyst
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id data-remediation-analyst-retail-banking-and-wealth-management-100003006551148
Job title is Data Remediation Analys

Job title is Major Bank: Data Warehouse (DataStage) Senior System Analysts/ System Analysts/ Analyst Programmers
company Major Bank: Data Warehouse (DataStage) Senior System Analysts/ System Analysts/ Analyst Programmers
 Got job role for this post Major Bank: Data Warehouse (DataStage) Senior System Analysts/ System Analysts/ Analyst Programmers
 Exp is 1 
 level is 1 
now to matching
Now start to craw individual job id senior-customer-data-analyst-well-known-institution-100003006537247
Job title is Senior Customer Data Analyst / Well-Known Institution
company Senior Customer Data Analyst / Well-Known Institution
 Got job role for this post Senior Customer Data Analyst / Well-Known Institution
 Exp is 8 
 level is 8 
now to matching
Now start to craw individual job id security-analyst-security-engineer-data-center-operation-100003006535722
Job title is Security Analyst / Security Engineer – Data Center Operation
company Security Analyst / Security Engineer – Data Center Operation
 Got

r                        846
bi                       744
excel                    422
sql                      354
sas                      147
java                     121
machine learning         115
hadoop                   110
api                      108
vba                       92
spark                     84
tableau                   83
business intelligence     81
linux                     80
gis                       71
scala                     69
unix                      58
mysql                     47
html                      44
aws                       37
javascript                36
nosql                     31
spss                      26
css                       24
php                       23
perl                      22
microsoft office          22
c#                        20
matlab                    17
html5                     17
azure                     16
tensorflow                16
mongodb                   16
deep learning             15
css3          

In [17]:
get_job_id('data scientist')

Now start to craw job search result P.1
data-scientist-100003006603427
data-scientist-100003006599779
lead-data-scientist-100003006598908
data-scientist-commercial-banking-100003006597216
data-scientists-ai-developer-and-researcher-big-data-engineer-100003006596791
data-scientist-100003006596532
lead-data-scientist-100003006595805
data-scientist-big-data-100003006593756
data-analyst-data-scientist-global-fintech-startup-100003006593695
data-scientist-100003006593633
senior-business-strategy-manager-business-strategy-manager-100003006593385
data-scientist-100003006591073
data-scientist-transaction-advisory-services-100003006588331
data-scientist-junior-data-scientist-100003006585948
data-scientist-computational-scientist-100003006585143
data-scientist-international-technology-consulting-frim-100003006585120
data-scientist-100003006585056
data-scientist-media-100003006581357
ai-data-scientist-100003006581181
customer-facing-data-scientist-100003006581010
data-scientist-100003006577448
da

Job title is Data Scientist
company Data Scientist
 Got job role for this post Data Scientist
 Exp is 3 
 level is 3 
now to matching
Now start to craw individual job id data-scientist-digital-platform-100003006577406
Job title is Data Scientist  - Digital Platform
company Data Scientist  - Digital Platform
 Got job role for this post Data Scientist  - Digital Platform
 Exp is 5 
 level is 5 
now to matching
Now start to craw individual job id data-scientist-100003006577405
Job title is Data Scientist
company Data Scientist
 Got job role for this post Data Scientist
 Exp is 3 
 level is 3 
now to matching
Now start to craw individual job id data-scientist-100003006576682
Job title is Data Scientist
company Data Scientist
 Got job role for this post Data Scientist
 Exp is Didnt specify 
 level is Didnt specify 
now to matching
Now start to craw individual job id data-scientist-100003006576063
Job title is Data Scientist
company Data Scientist
 Got job role for this post Data Scientist
 

Job title is Data Scientist (NLP) - Buy Side
company Data Scientist (NLP) - Buy Side
 Got job role for this post Data Scientist (NLP) - Buy Side
 Exp is 6 
 level is 6 
now to matching
Now start to craw individual job id manager-supply-chain-analytics-100003006532023
Job title is Manager - Supply Chain Analytics (Data Scientist)
company Manager - Supply Chain Analytics (Data Scientist)
 Got job role for this post Manager - Supply Chain Analytics (Data Scientist)
 Exp is 5 
 level is 5 
now to matching
Now start to craw individual job id senior-data-scientist-100003006530512
Job title is Senior Data Scientist
company Senior Data Scientist
 Got job role for this post Senior Data Scientist
 Exp is 1 
 level is 1 
now to matching
It takes 19.744305849075317 to crawd all the jobs info
There are total 51 jobs in this category


r                        897
bi                       792
excel                    442
sql                      381
machine learning         159
sas                      156
java                     136
hadoop                   130
api                      115
spark                    104
vba                       93
tableau                   92
business intelligence     84
linux                     82
gis                       77
scala                     74
unix                      58
mysql                     50
html                      46
javascript                42
aws                       37
nosql                     36
spss                      27
tensorflow                24
deep learning             24
php                       24
css                       24
perl                      22
microsoft office          22
matlab                    22
c#                        21
mongodb                   17
html5                     17
azure                     16
keras         